In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
import pickle
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares 

d:\recommend\Recommendation_Systems\RS_virt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Exploratory Data Analysis

In [ ]:


# Load raw CSV files
events_df = pd.read_csv("events.csv")
item_properties_1 = pd.read_csv("item_properties_part1.1.csv")
item_properties_2 = pd.read_csv("item_properties_part2.csv")
category_tree = pd.read_csv("category_tree.csv")

# Merge item properties (since it's split into two parts)
item_properties = pd.concat([item_properties_1, item_properties_2])

# Display basic info
print("Events Dataset Info:")
events_df.info()
print("\nItem Properties Dataset Info:")
item_properties.info()
print("\nCategory Tree Dataset Info:")
category_tree.info()
